In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("DeltaJupyter")
    .master("spark://spark-master:7077")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

spark

spark.conf.set(
  "spark.databricks.delta.retentionDurationCheck.enabled", "false"
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/05 20:32:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = [
    (1, "Veera", "Engineering"),
    (2, "Asha", "Finance")
]

df = spark.createDataFrame(data, ["id", "name", "dept"])

df.write.format("delta") \
  .mode("overwrite") \
  .save("/opt/spark/work-dir/data/delta/employees")

In [7]:
spark.read.format("delta") \
  .load("/opt/spark/work-dir/data/delta/employees") \
  .show()

+---+-----+-----------+
| id| name|       dept|
+---+-----+-----------+
|  1|Veera|Engineering|
|  2| Asha|    Finance|
+---+-----+-----------+



In [9]:
ls data/delta/employees/_delta_log/

00000000000000000000.json


In [10]:
new_data = [(3, "Raj", "Risk")]
spark.createDataFrame(new_data, ["id","name","dept"]) \
  .write.format("delta") \
  .mode("append") \
  .save("/opt/spark/work-dir/data/delta/employees")

In [11]:
ls data/delta/employees/_delta_log

00000000000000000000.json  00000000000000000001.json


In [12]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(
    spark, "/opt/spark/work-dir/data/delta/employees"
)

delta_table.history().show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-12-19 03:34:05.083|NULL  |NULL    |WRITE    |{mode -> Append, partitionBy -> []}   |NULL|NULL    |NULL     |0          |Serializable  |true         |{numFiles -> 2, nu

In [13]:
spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("/opt/spark/work-dir/data/delta/employees") \
  .show()

+---+-----+-----------+
| id| name|       dept|
+---+-----+-----------+
|  1|Veera|Engineering|
|  2| Asha|    Finance|
+---+-----+-----------+



In [14]:
bad_df = spark.createDataFrame(
    [(4, "Bad", "Ops", 999)],
    ["id", "name", "dept", "salary"]
)

bad_df.write.format("delta") \
  .mode("append") \
  .save("/opt/spark/work-dir/data/delta/employees")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 463bfa90-7451-4fb5-b763-56023e373d58).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- dept: string (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- dept: string (nullable = true)
-- salary: long (nullable = true)

         

In [15]:
bad_df.write.format("delta") \
  .option("mergeSchema", "true") \
  .mode("append") \
  .save("/opt/spark/work-dir/data/delta/employees")

In [16]:
spark.read.format("delta") \
  .load("/opt/spark/work-dir/data/delta/employees") \
  .show()

+---+-----+-----------+------+
| id| name|       dept|salary|
+---+-----+-----------+------+
|  4|  Bad|        Ops|   999|
|  1|Veera|Engineering|  NULL|
|  2| Asha|    Finance|  NULL|
|  3|  Raj|       Risk|  NULL|
+---+-----+-----------+------+



In [17]:
from delta.tables import DeltaTable

updates = spark.createDataFrame(
    [(2, "Asha", "HR"), (5, "Nina", "Legal")],
    ["id", "name", "dept"]
)

delta_table.alias("t") \
  .merge(
      updates.alias("s"),
      "t.id = s.id"
  ) \
  .whenMatchedUpdate(set={"dept": "s.dept"}) \
  .whenNotMatchedInsertAll() \
  .execute()

25/12/19 03:34:50 ERROR MergeIntoCommand: Fatal error in MERGE with materialized source in attempt 1.
org.apache.spark.sql.delta.DeltaAnalysisException: [DELTA_SCHEMA_CHANGE_SINCE_ANALYSIS] The schema of your Delta table has changed in an incompatible way since your DataFrame
or DeltaTable object was created. Please redefine your DataFrame or DeltaTable object.
Changes:
Latest schema has additional field(s): salary
	at org.apache.spark.sql.delta.DeltaErrorsBase.schemaChangedSinceAnalysis(DeltaErrors.scala:553)
	at org.apache.spark.sql.delta.DeltaErrorsBase.schemaChangedSinceAnalysis$(DeltaErrors.scala:540)
	at org.apache.spark.sql.delta.DeltaErrors$.schemaChangedSinceAnalysis(DeltaErrors.scala:3203)
	at org.apache.spark.sql.delta.commands.MergeIntoCommand.$anonfun$runMerge$2(MergeIntoCommand.scala:89)
	at org.apache.spark.sql.delta.commands.MergeIntoCommand.$anonfun$runMerge$2$adapted(MergeIntoCommand.scala:83)
	at org.apache.spark.sql.delta.DeltaLog.withNewTransaction(DeltaLog.scala:2

AnalysisException: [DELTA_SCHEMA_CHANGE_SINCE_ANALYSIS] The schema of your Delta table has changed in an incompatible way since your DataFrame
or DeltaTable object was created. Please redefine your DataFrame or DeltaTable object.
Changes:
Latest schema has additional field(s): salary

In [24]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(
    spark,
    "/opt/spark/work-dir/data/delta/employees"
)

updates = spark.createDataFrame(
    [(2, "Asha", "HR"), (5, "Nina", "Legal")],
    ["id", "name", "dept"]
)

delta_table.alias("t") \
  .merge(
      updates.alias("s"),
      "t.id = s.id"
  ) \
  .whenMatchedUpdate(set={
      "dept": "s.dept",
      "name": "s.name"
  }) \
  .whenNotMatchedInsert(values={
      "id": "s.id",
      "dept": "s.dept",
      "name": "s.name",
      "salary": "NULL"
  }) \
  .execute()


In [25]:
spark.read.format("delta") \
  .load("/opt/spark/work-dir/data/delta/employees") \
  .orderBy("id") \
  .show()

+---+-----+-----------+------+
| id| name|       dept|salary|
+---+-----+-----------+------+
|  1|Veera|Engineering|  NULL|
|  2| Asha|         HR|  NULL|
|  3|  Raj|       Risk|  NULL|
|  4|  Bad|        Ops|   999|
|  5| Nina|      Legal|  NULL|
+---+-----+-----------+------+



In [26]:
delta_table.delete("dept = 'Risk'")
delta_table.update(
    condition="dept = 'HR'",
    set={"dept": "'People'"}
)

In [27]:
spark.read.format("delta") \
  .load("/opt/spark/work-dir/data/delta/employees") \
  .orderBy("id") \
  .show()

+---+-----+-----------+------+
| id| name|       dept|salary|
+---+-----+-----------+------+
|  1|Veera|Engineering|  NULL|
|  2| Asha|     People|  NULL|
|  4|  Bad|        Ops|   999|
|  5| Nina|      Legal|  NULL|
+---+-----+-----------+------+



In [28]:
from delta.tables import DeltaTable

dt = DeltaTable.forPath(
    spark, "/opt/spark/work-dir/data/delta/employees"
)

dt.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation|operationMetrics                                                                                                                                                                                                                                                                                                                         

In [29]:
spark.sql("""
OPTIMIZE delta.`/opt/spark/work-dir/data/delta/employees`
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [32]:
spark.sql("""
VACUUM delta.`/opt/spark/work-dir/data/delta/employees` RETAIN 0 HOURS
""")

Deleted 12 files and directories in a total of 1 directories.


DataFrame[path: string]

In [33]:
dt.history().select("version","operation","operationMetrics").show(truncate=False)

+-------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|operation   |operationMetrics                                                                                                                                                                                                                                                                                                                   

In [34]:
spark.sql("""
CREATE TABLE employees_delta
USING DELTA
LOCATION '/opt/spark/work-dir/data/delta/employees'
""")

spark.sql("SELECT * FROM employees_delta").show()

+---+-----+-----------+------+
| id| name|       dept|salary|
+---+-----+-----------+------+
|  4|  Bad|        Ops|   999|
|  2| Asha|     People|  NULL|
|  5| Nina|      Legal|  NULL|
|  1|Veera|Engineering|  NULL|
+---+-----+-----------+------+

